In [ ]:
! wget http://ftp.unicamp.br/pub/apache/lucene/solr/8.2.0/solr-8.2.0.zip -q
! unzip -qq solr-8.2.0.zip && rm solr-8.2.0.zip
! ./solr-8.2.0/bin/solr start -c

In [ ]:
'''! ./solr-8.2.0/bin/solr create -c cabot -p 8983
import pysolr
# Create a client instance. The timeout and authentication options are not required.
solr = pysolr.Solr('http://localhost:8983/solr/', always_commit=True, [timeout=10], [auth=<type of authentication>])

# Note that auto_commit defaults to False for performance. You can set
# `auto_commit=True` to have commands always update the index immediately, make
# an update call with `commit=True`, or use Solr's `autoCommit` / `commitWithin`
# to have your data be committed following a particular policy.

# Do a health check.
solr.ping()'''

In [22]:
! ./solr-8.2.0/bin/solr create -c mesh -p 8983
import pysolr
solr_server_url = 'http://localhost:8983/solr/'
solr_collection = "mesh"
solr = pysolr.Solr(solr_server_url + solr_collection)

         To turn off: bin/solr config -c mesh -p 8983 -action set-user-property -property update.autoCreateFields -value false

ERROR: 
Collection 'mesh' already exists!
Checked collection existence using Collections API command:
http://localhost:8983/solr/admin/collections?action=list



In [23]:

import re
import string
import pandas as pd
from functools import reduce
from math import log

In [24]:
mesh_read = open("c2020.bin", "r")

In [19]:
'''i = -1;
vet = [""]
for linha in mesh_read:
    if(linha == "*NEWRECORD\n"):
        i+=1
        vet.append("")
    vet[i]+=linha
    
vet[0]'''

'i = -1;\nvet = [""]\nfor linha in mesh_read:\n    if(linha == "*NEWRECORD\n"):\n        i+=1\n        vet.append("")\n    vet[i]+=linha\n    \nvet[0]'

In [25]:
i = -1;
vet = []
for linha in mesh_read:
    if(linha == "*NEWRECORD\n"):
        i+=1
        vet.append([])
        algo = 0
        algo2 = 0
        algo3 = 0
        a = 0
        vet[i] = ["", "NM", "", "SY", "", "PI", "", "PA", "", "NO", ""]
    else:
        l = linha.split("= ")
        if("NM" in l[0] and "NM_TH" not in l[0]):
            vet[i][2]+=l[1]
        elif("SY" in l[0]):
            l2 = l[1].split("|")
            vet[i][4]+=l2[0]
        elif("PI" in l[0]):
            vet[i][6]+=l[1]
        elif("PA" in l[0]):
            vet[i][8]+=l[1]
        elif("NO" in l[0]):
            vet[i][10]+=l[1]
    vet[i][0]+=linha
    
vet[0]

['*NEWRECORD\nRECTYPE = C\nNM = bevonium\nRN = 34B0471E08\nRR = 33371-53-8 (bevonium)\nRR = 35517-05-6\nRR = 5205-82-3 (bevoinum methylsulfate)\nRR = UWC15E373Z\nSY = 2-(hydroxymethyl)-N,N-dimethylpiperidinium benzilate|T109|T121|EQV|NLM (2017)|180918|abbdef\nSY = Acabel|T109|T121|TRD|NRW|NLM (1971)|abbcde\nSY = CG 201|T109|T121|LAB|NRW|NLM (1971)|abbcde\nSY = bevonium methyl sulfate|T109|T121|NRW|FDA SRS (2015)|NLM (1971)|abbdee\nSY = bevonium methylsulfate|T109|T121|NRW|NLM (1971)|990818|abbdef\nSY = bevonium metilsulfate|T109|T121|NRW|INN (19XX)|150601|abbdef\nSY = bevonium sulfate (1:1)|T109|T121|NRW|NLM (1971)|990818|abbdef\nSY = piribenzil methyl sulfate|T109|T121|NRW|NLM (1971)|abbde\nHM = *Benzilates\nPI = PIPERIDINES (71-81)\nSO = S Afr Med J 50(1):4;1976\nNM_TH = FDA SRS (2013)\nNM_TH = NLM (1971)\nST = T109\nST = T121\nPA = Anti-Inflammatory Agents, Non-Steroidal\nFR = 1\nNO = structure given in first source\nDA = 19710101\nMR = 20180924\nUI = C000002\n\n',
 'NM',
 'bevonium

In [ ]:
for i in range(len(vet)):
    indice = "doc_"+str(i)
    solr.add([{
        "ID": indice,
        "Body": vet[i][0],
        "Name": vet[i][2],
        "Synonym": vet[i][4],
        "Previous Indexing": vet[i][6],
        "Pharmacological Action": vet[i][8],
        "Note": vet[i][10],    
    },
        ],commit=True)

In [ ]:
def procura(palavra):
    results = solr.search('Name:{}'.format(palavra))
    results += solr.search('Synonym:{}'.format(palavra))
    results += solr.search('Previous Indexing:{}'.format(palavra))
    results += solr.search('Pharmacological Action:{}'.format(palavra))
    results += solr.search('Note:{}'.format(palavra))
    return results

In [7]:
results = solr.search('body:"Calcimycin"')

print("Found {0} documents".format(len(results)))


Found 0 documents


In [ ]:
for resutados in results:
    print(resultados)

In [ ]:
mesh_read.close()